### Assumptions for the independent t test

1. Underlying population variances that the two samples are drawn from are equal
2. Underlying populations are normally distributed

In [1]:
import pandas as pd
import io
import requests
from IPython.core.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline

pd.options.display.max_columns = 50

*From Section 8.4 of* ***Fundamentals of Biostatistics by Bernard Rosner, 8th Edition***

Hypertension Suppose a sample of eight 35- to 39-year-old nonprenant, premeno- pausal OC users who work in a company and have a mean systolic blood pres- sure (SBP) of 132.86 mm Hg and sample standard deviation of 15.34 mm Hg are identified. A sample of 21 nonpregnant, premenopausal, non-OC users in the same age group are similarly identified who have mean SBP of 127.44 mm Hg and sample standard deviation of 18.23 mm Hg. What can be said about the underlying mean difference in blood pressure between the two groups?


Assume SBP is normally distributed in the first group with mean μ1 and variance σ1 and in the second group with mean μ2 and variance σ2. We want to test the hypothesis H0: μ1 = μ2 vs. H1: μ1 ≠ μ2. Assume in this section that the underlying variances in the two groups are the same (that is, σ12 = σ2 = σ2). The means and variances in the two samples are denoted by x1 x2 , s12 , s2 , respectively.

### Estimating the population variance by combining the sample variances

Calculate a weighted average of the sample variances.

**pooled estimate of the variance** 


INSERT EQUATION HERE

In [12]:
from scipy.stats import ttest_ind, ttest_ind_from_stats

In [13]:
mean1 = 132.86
std1 = 15.34
nobs1 = 8
mean2 = 127.44
std2 = 18.23
nobs2 = 21

In [14]:
print(mean2 - mean1)

-5.420000000000016


In [15]:
statistic, pvalue = ttest_ind_from_stats(mean1, std1, nobs1, mean2, std2, nobs2)

print(statistic, pvalue)

0.7443175718105018 0.46311371461667283


Even though the mean difference between the two samples in both experiments are similar, we get wildly different p values - the paired t-test example rejects H0, whereas the independent t-test example accepts H0.

*What this proves for me is that paired t-test is much more sensitive to differences than the independent t-test. In other words, you need a much more significant difference between two independent samples to detect a difference than you would if the samples were paired. This makes intuitive sense because for paired data, we've (supposedly) accounted for many confounding variables, so we can attribute much more of the variance in the measurement of interest between the two (paired) samples to the intervention.*

### Calculating the 95% CI 

the combined variance is just a weighted sum of the individual variances:<br>
$s^{2} = { \left[(n_1 - 1)s_1^2 + (n_2 - 1)s_2^2\right] / (n_1 + n_2 - 2)}$ <br>

take the square root to get the combined standard deviation:<br>
$s = \sqrt{ \left[(n_1 - 1)s_1^2 + (n_2 - 1)s_2^2\right] / (n_1 + n_2 - 2)}$<br>


As usual, the CI is calculated by taking the average (in this case the difference in means from the two samples) and adding/subtracting the t statistic mulitplied by the standard deviation of the sampling distribution. (for a full derivation, see page 287 from the text).

95% CI = $(\hat{x_1} - \hat{x_2} − t_{n_1+n_2-2,1−α/2} * s /\sqrt{1/n_1 + 1/n_2},  \hat{x_1} - \hat{x_2} + t_{n_1+n_2-2,1−α/2} * s/\sqrt{1/n_1 + 1/n_2})$

In [16]:
def ci_ind(a, b, alpha=0.95):
    '''Calculates the confidence interval corresponding to significance {alpha} for two independent
    samples a and b.
    
    Parameters:
        a -- {np.array}
        b -- {np.array}
        alpha -- {float}
    
    Returns:
        mean_diff -- {float}
        ci -- {tuple}
    
    '''
    
    # make sure input vectors are the same length
    assert(len(a) == len(b))
    n1 = len(a)
    n2 = len(b)
    
    s1 = np.std(a)
    s2 = np.std(b)
    var = ((n1 - 1)*s1 + (n2-1)*s2) / (n1 + n2 - 2)
    std = np.sqrt(var)
    
    # calculate mean difference
    diff = b.mean() - a.mean()
    

    # generate a t distributed random variable 
    dof = n - 1 # calculate degrees of freedom
    rv = scipy.stats.t(dof) # instantiate A Student’s t continuous random variable
    t = rv.ppf((1 - alpha) / 2)
    t = np.abs(t) 
    
    # calculate ci
    ci = (diff - t * (std / np.sqrt((1/n1) + (1/n2))), diff + t * (std / np.sqrt((1/n1) + (1/n2))))
    
    return  mean_diff, ci